# Basic agentic workflows, for a marketing agent use case

This notebook walks through 3 simple multi-LLM workflows, with the use case of a marketing agent being the guiding example throughout.

1. **Prompt-chaining**: Decompose a task into sequential subtasks, where eaech step builds on previous results.
2. **Parallelization**: Distributes independent subtasks across multiple LLMs or concurrent processing.
3. **Routing**: Dynamically selects specialized LLM paths based on input characteristics.

Our motivating example will be creating a restaurant marketing AI agent that writes marketing copy across various mediums (email blast, blog post, social media). We'll see how each of these methods approaches this task.

#### Step 0: some setup

In [14]:
import sys
import os

project_root = os.path.abspath(os.path.dirname(os.getcwd()))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
print("PYTHONPATH set to:", project_root)

PYTHONPATH set to: /Users/mark/Documents/work/demos/ai_agent_learning_examples


In [22]:
import sys
print("Python executable:", sys.executable)
print("\nPython path:")
for i, path in enumerate(sys.path):
    if i >= 1:
        break
    print(f"  {path}")

print(f"\nCurrent working directory: {os.getcwd()}")
print(f"Directory contents:")
import os
for i, item in enumerate(os.listdir('.')):
    if i >= 1:
        break
    print(f"  {i}: {item}")


Python executable: /Users/mark/anaconda3/envs/ai_agent_learning_examples/bin/python

Python path:
  /Users/mark/Documents/work/demos/ai_agent_learning_examples

Current working directory: /Users/mark/Documents/work/demos/ai_agent_learning_examples/marketing_agent_examples
Directory contents:
  0: models.py


## 1. Prompt-chaining

This method breaks down a task into a series of sequential subtasks, where each step builds on previous results.

For our example, we can break down the marketing AI agent chain into each of these steps:

1. Create the idea to promote.
2. Evaluate and score the ideas. Then pick one to continue with.
3. Expand the idea into a blog post.
4. Score/evaluate the blog post.
5. Summarize blog post for email.
6. Score/evaluate the email-formatted result.
7. Create social media captions.
8. Score/evaluate the social media captions.

Let's do each of these in turn.

### 1a. Create ideas for what to promote.

Here, let's create a first pass of our prompt for our idea generation step.

In [10]:
idea_generation_prompt = """
You are a marketing agent. You are given a product and a target audience.
You will generate a list of ideas for a marketing campaign.

Client: American-style brunch restaurant.

Product: Brunch menu. Things we want to promote include:
- $9.99 full breakfast with coffee. Features 2 eggs, 3 pieces of bacon
(or choice of meat), potatoes, fresh fruit, and coffee.
- Hearty Mediterranean wrap for lunch
- Popular $20.99 Sunday brunch buffet featuring full breakfast options
(eggs, pancakes, waffles, bacon, sausage, hash browns, omelettes)
and lunch options (salad bar, meats, etc.).

Target audience:
    - Families
    - Health-minded individuals
    - Brunch group gatherings
"""

Let's actually pass this into an LLM to improve the prompt. We can use an LLM for this prompt improving task, and there are explicit tools like Claude's [prompt improver](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/prompt-improver) made for this task alone.

Here's the improved version that ChatGPT gave me:


In [11]:
idea_generation_prompt = """
You are an expert marketing agent helping a neighborhood American-style brunch restaurant design a creative and targeted marketing campaign. You will be given product offerings and a target audience. Your job is to generate 5–7 campaign ideas that highlight the value of the offerings and appeal to the specific interests of the audience segments.

Client: Independent American-style brunch restaurant known for quality and community.

Product offerings to highlight:
1. Budget-friendly $9.99 full breakfast combo — includes 2 eggs, choice of meat (bacon, sausage, or veggie patty), breakfast potatoes, fresh fruit, and coffee.
2. Hearty and nutritious Mediterranean wrap — packed with veggies and lean protein, great for health-conscious diners.
3. Popular $20.99 all-you-can-eat Sunday brunch buffet — includes full breakfast classics (eggs, pancakes, waffles, bacon, sausage, hash browns, omelets) and lunch options (salad bar, seasonal meats, and sides).

Target audience:
- Families looking for weekend outings
- Health-conscious individuals seeking nutritious, flavorful options
- Social brunch groups celebrating milestones or gathering casually

Output:
- A list of 5-7 specific marketing campaign ideas
- Each idea should include a title and a 2-3 sentence explanation
- Tailor each idea to one or more of the audience segments
- Highlight value, experience, or emotional appeal
"""

Let's also make sure that our prompt conforms to a specific format.


In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage


In [17]:
from marketing_agent_examples.models import ProposedIdeasWrapper

parser = PydanticOutputParser(pydantic_object=ProposedIdeasWrapper)

In [23]:
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="""
        {idea_generation_prompt}
        {format_instructions}
    """,
    input_variables=[],
    partial_variables={
        "format_instructions": format_instructions,
        "idea_generation_prompt": idea_generation_prompt
    }
)

In [24]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

/var/folders/9g/nhh_jmv17sd2604y43zdctzw0000gn/T/ipykernel_93715/940076340.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)


In [25]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content=prompt.format())
]

Let's try this out and see what we get.

In [26]:
response = llm(messages)
parsed = parser.parse(response.content)

/var/folders/9g/nhh_jmv17sd2604y43zdctzw0000gn/T/ipykernel_93715/3050779672.py:1: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(messages)


In [35]:
generated_ideas: ProposedIdeasWrapper = parsed.ideas

for i, idea in enumerate(generated_ideas, 1):
    print(f"\n=== Idea {i} ===")
    print(f"Idea: {idea.idea}")
    print(f"Audience: {idea.audience}")
    print(f"Campaign Message: {idea.campaign_message}")
    print("=" * 50)


=== Idea 1 ===
Idea: Family Fun Brunch Day
Audience: Families looking for weekend outings
Campaign Message: Create lasting memories with our budget-friendly $9.99 full breakfast combo, perfect for families!

=== Idea 2 ===
Idea: Healthy Start Sundays
Audience: Health-conscious individuals seeking nutritious, flavorful options
Campaign Message: Kickstart your week with our nutritious Mediterranean wrap, packed with fresh veggies and lean protein!

=== Idea 3 ===
Idea: Brunch & Celebrate
Audience: Social brunch groups celebrating milestones or gathering casually
Campaign Message: Celebrate life’s moments with our all-you-can-eat Sunday brunch buffet, perfect for gatherings!

=== Idea 4 ===
Idea: Brunch Buddy Loyalty Program
Audience: Families looking for weekend outings and social brunch groups
Campaign Message: Join our Brunch Buddy Loyalty Program and earn rewards for every visit!

=== Idea 5 ===
Idea: Weekend Brunch Specials
Audience: Families looking for weekend outings and health-c

### 1b. Evaluate the created ideas.

Now that we have all of these ideas, let's ask the LLM to evaluate the ideas and help us pick the best one.

In [33]:
from models import IdeaEvaluationOutput
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=IdeaEvaluationOutput)

idea_evaluation_instructions = parser.get_format_instructions()

In [34]:
idea_evaluation_prompt = PromptTemplate(
    template="""
You are a senior marketing strategist evaluating proposed campaign ideas for an American-style brunch restaurant.

Evaluate each idea on the following criteria (score from 0-5):
- **Audience Fit**: Does it match the needs or preferences of the specified audience?
- **Clarity**: Is the idea easy to understand and well-articulated?
- **Creativity**: How original or compelling is the campaign concept?
- **Channel Suitability**: Does the distribution method or concept fit real-world marketing channels (e.g., Instagram, flyers, email)?

Also include brief **comments** on the strengths or weaknesses of each idea.

Here are the proposed ideas to review:
{idea_text}

{evaluation_instructions}
""",
    input_variables=["idea_text"],
    partial_variables={"evaluation_instructions": idea_evaluation_instructions}
)

Let's now take our previous ideas from above and pass them into this prompt.

In [39]:
from marketing_agent_examples.models import ProposedIdea

def format_ideas_for_evaluation(ideas: list[ProposedIdea]) -> str:
    result = []
    for i, idea in enumerate(ideas, 1):
        result.append(f"""
            Idea {i}:
            Audience: {idea.audience}
            Name: {idea.idea}
            Message: {idea.campaign_message}
            Concept: {idea.concept}
        """
        )
    return "\n\n".join(result)

In [40]:
idea_text = format_ideas_for_evaluation(generated_ideas)

In [48]:
print(idea_text[:1000])


            Idea 1:
            Audience: Families looking for weekend outings
            Name: Family Fun Brunch Day
            Message: Create lasting memories with our budget-friendly $9.99 full breakfast combo, perfect for families!
            Concept: Host a special Family Fun Day every Saturday where kids eat free with the purchase of an adult breakfast combo. Include fun activities like face painting or a small petting zoo to enhance the family outing experience.
        


            Idea 2:
            Audience: Health-conscious individuals seeking nutritious, flavorful options
            Name: Healthy Start Sundays
            Message: Kickstart your week with our nutritious Mediterranean wrap, packed with fresh veggies and lean protein!
            Concept: Promote a 'Healthy Start Sunday' where customers can enjoy a discount on the Mediterranean wrap when they order a drink. Share health tips and recipes on social media to engage health-conscious diners.
        


  

In [51]:
from models import IdeaEvaluationOutput, IdeaEvaluation

In [49]:
def evaluate_ideas(ideas: ProposedIdeasWrapper) -> IdeaEvaluationOutput:
    idea_evaluation_full_prompt = idea_evaluation_prompt.format(
        idea_text=format_ideas_for_evaluation(ideas)
    )
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=idea_evaluation_full_prompt)
    ]
    response = llm.invoke(messages)
    return parser.parse(response.content)

In [50]:
evaluated_ideas: IdeaEvaluationOutput = evaluate_ideas(parsed.ideas)

Let's now review the ideas

In [55]:
for i, evaluation in enumerate(evaluated_ideas.evaluations, 1):
    print(f"Idea {i}: {evaluation.idea_name}")
    print(f"Audience Fit: {evaluation.audience_fit}")
    print(f"Clarity: {evaluation.clarity}")
    print(f"Creativity: {evaluation.creativity}")
    print(f"Channel Suitability: {evaluation.channel_suitability}")
    print(f"Comments: {evaluation.comments}")
    print("=" * 50)

Idea 1: Family Fun Brunch Day
Audience Fit: 5
Clarity: 5
Creativity: 4
Channel Suitability: 5
Comments: This idea perfectly targets families looking for weekend outings with a clear message and engaging activities. The concept is creative and well-suited for social media promotion.
Idea 2: Healthy Start Sundays
Audience Fit: 5
Clarity: 5
Creativity: 4
Channel Suitability: 4
Comments: This campaign aligns well with health-conscious individuals and is clearly articulated. The idea of sharing health tips is creative, but the execution may be limited to social media and in-restaurant promotions.
Idea 3: Brunch & Celebrate
Audience Fit: 5
Clarity: 5
Creativity: 4
Channel Suitability: 5
Comments: This idea effectively targets social groups celebrating milestones. The concept is clear and encourages social media engagement, making it suitable for various marketing channels.
Idea 4: Brunch Buddy Loyalty Program
Audience Fit: 4
Clarity: 5
Creativity: 3
Channel Suitability: 5
Comments: While thi

Let's sort the ideas by score and then pick the highest-scoring idea.

In [58]:
def pick_highest_scoring_idea(
    ideas: ProposedIdeasWrapper,
    evaluations: IdeaEvaluationOutput
) -> tuple[ProposedIdea, IdeaEvaluation]:
    """
    Given a list of ideas and their evaluations, return the highest scoring idea and its evaluation.

    Returns:
        A tuple of (ProposedIdea, IdeaEvaluation)
    """
    best_score = -1
    best_idea = None
    best_evaluation = None

    idea_map = {idea.idea: idea for idea in ideas}

    for evaluation in evaluations.evaluations:
        total_score = (
            evaluation.audience_fit +
            evaluation.clarity +
            evaluation.creativity +
            evaluation.channel_suitability
        )

        if total_score > best_score:
            best_score = total_score
            best_idea = idea_map.get(evaluation.idea_name)
            best_evaluation = evaluation

    if best_idea is None or best_evaluation is None:
        raise ValueError("No matching idea found for highest scoring evaluation.")

    return best_idea, best_evaluation

In [60]:
best_idea, best_evaluation = pick_highest_scoring_idea(
    ideas=generated_ideas,
    evaluations=evaluated_ideas
)

In [61]:
print(f"Best idea: {best_idea.idea}")
print(f"Best evaluation:\n")
print(f"Audience Fit: {best_evaluation.audience_fit}")
print(f"Clarity: {best_evaluation.clarity}")
print(f"Creativity: {best_evaluation.creativity}")
print(f"Channel Suitability: {best_evaluation.channel_suitability}")
print(f"Comments: {best_evaluation.comments}")

Best idea: Brunch with a Cause
Best evaluation:

Audience Fit: 5
Clarity: 5
Creativity: 5
Channel Suitability: 5
Comments: This campaign is highly engaging and appeals to social groups while supporting local charities. The clarity and creativity are strong, and it fits well with various marketing channels.


Great! We now have a flow to (1) create new marketing ideas and (2) review the ideas and score them so that we can pick the best one.

Some possible concepts we can explore later on are:
1. Rewriting poor-rated ideas.
2. Generating multiple variations of high-potential ideas to test tone, format, or messaging.
3. See which ideas end up doing well, and then figure out what the best criteria to filter on are based on which of those ideas perform well.
4. Honing the tone and branding based on past examples of copywriting.


### 1c. Create an SEO-optimized blog post out of the idea.

Given this idea, let's now create an SEO-optimized blog post.

In [65]:
# optional code snippet to reload the models module if the
# BlogPost import doesn't work. Jupyter notebooks can cache
# module imports.
import importlib
import marketing_agent_examples.models

# Force reload the module
importlib.reload(marketing_agent_examples.models)


<module 'marketing_agent_examples.models' from '/Users/mark/Documents/work/demos/ai_agent_learning_examples/marketing_agent_examples/models.py'>

In [66]:
from marketing_agent_examples.models import BlogPost
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=BlogPost)

blog_post_instructions = parser.get_format_instructions()

In [67]:
blog_prompt = PromptTemplate(
    template="""
        You are a content marketer creating a blog post for an American-style brunch restaurant.

        Your job is to write an SEO-optimized blog post based on the following campaign idea:

        <idea>
        Name: {idea_name}
        Target Audience: {audience}
        Campaign Message: {message}
        Concept: {concept}
        </idea>

        You must optimize this blog post for:
        - **Search Engine Visibility**: Use high-intent brunch-related keywords naturally throughout.
        - **Click-Through Rate**: Title and excerpt should be emotionally compelling, clear, and benefit-driven.
        - **Engagement**: Structure the content with subheadings, short paragraphs, and a clear flow.
        - **Audience Fit**: Make the tone match the specified audience. You may include humor, warmth, or health-focused language if appropriate.

        Output Format:
        {format_instructions}

        Success is defined as:
        - The title is both SEO-relevant and emotionally appealing.
        - The excerpt would make a reader want to click through.
        - The content clearly communicates the campaign idea while being useful, fun, and on-brand.
        - Keywords are relevant and well-targeted to brunch-goers and local audiences.
""",
    input_variables=["idea_name", "audience", "message", "concept"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [68]:
def create_blog_post(idea: ProposedIdea) -> BlogPost:
    blog_post_full_prompt = blog_prompt.format(
        idea_name=idea.idea,
        audience=idea.audience,
        message=idea.campaign_message,
        concept=idea.concept
    )
    messages = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=blog_post_full_prompt)
    ]
    response = llm.invoke(messages)
    return parser.parse(response.content)

In [69]:
blog_post: BlogPost = create_blog_post(best_idea)

In [70]:
print(f"Best idea: {best_idea.idea}")
print("Blog post:\n")
print(f"Title: {blog_post.title}")
print(f"Slug: {blog_post.slug}")
print(f"Excerpt: {blog_post.excerpt}")
print(f"Content: {blog_post.content}")
print(f"Keywords: {blog_post.keywords}")

Best idea: Brunch with a Cause
Blog post:

Title: Brunch with a Cause: Enjoy Delicious Food While Supporting Local Charities
Slug: brunch-with-a-cause
Excerpt: Join us for our monthly 'Brunch with a Cause' events! Gather your friends, indulge in an all-you-can-eat buffet, and make a difference in our community.
Content: ### Gather for Good: Brunch with a Cause

Are you ready to elevate your brunch game? At our restaurant, we believe that brunch is not just a meal; it’s a celebration of friendship, community, and giving back. That’s why we’re excited to introduce our **Brunch with a Cause** events! 

Every month, we invite you and your social brunch groups to join us for a delightful all-you-can-eat buffet, where a portion of the proceeds goes directly to a local charity. It’s the perfect way to enjoy a leisurely brunch while making a positive impact in our community.

### Why Brunch with a Cause?
Brunch is the ideal time to gather with friends, celebrate milestones, or simply enjoy a c

### 1d. Evaluate the blog post.

Now that we have a blog post, let's ask the LLM to evaluate our blog post and score it.

Let's score our blog post on these criteria:

| Criterion                  | Description                                          |
| -------------------------- | ---------------------------------------------------- |
| **SEO Optimization** (0–5) | Keyword use, metadata presence, search relevance     |
| **Clickability** (0–5)     | Emotional hook + clarity of title and excerpt        |
| **Readability** (0–5)      | Structure, formatting, paragraph length, headings    |
| **Audience Fit** (0–5)     | Language and tone tailored to the intended reader    |
| **Content Quality** (0–5)  | Informative, engaging, clear                         |
| **Comments**               | Summary of strengths and suggestions for improvement |


In [74]:
# optional code snippet to reload the models module if the
# below import doesn't work. Jupyter notebooks can cache
# module imports.
import importlib
import marketing_agent_examples.models

# Force reload the module
importlib.reload(marketing_agent_examples.models)


<module 'marketing_agent_examples.models' from '/Users/mark/Documents/work/demos/ai_agent_learning_examples/marketing_agent_examples/models.py'>

In [76]:
from marketing_agent_examples.models import BlogPostEvaluation

parser = PydanticOutputParser(pydantic_object=BlogPostEvaluation)

blog_post_evaluation_instructions = parser.get_format_instructions()

In [77]:
blog_post_evaluation_prompt = PromptTemplate(
    template="""
        You are a senior SEO content editor evaluating a blog post for an American-style brunch restaurant.

        Evaluate the post based on the following 5 criteria (0-5 scale):
        - **SEO Optimization**: Does the post use relevant keywords naturally? Is the title and excerpt search-friendly? Are metadata fields filled?
        - **Clickability**: Does the title and excerpt compel users to click? Is there emotional or benefit-driven language?
        - **Readability**: Is the post well-structured with good formatting (headings, paragraph length, etc)?
        - **Audience Fit**: Is the tone and language tailored to the intended audience?
        - **Content Quality**: Is it engaging, informative, and clear?

        Also provide 2-3 sentences of comments on strengths and areas for improvement.

        Blog post to evaluate:

        Title: {title}
        Slug: {slug}
        Excerpt: {excerpt}
        Content: {content}
        Keywords: {keywords}

        {format_instructions}
""",
    input_variables=["title", "slug", "excerpt", "content", "keywords"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [78]:
def evaluate_blog_post(blog_post: BlogPost):
    prompt_text = blog_post_evaluation_prompt.format(
        title=blog_post.title,
        slug=blog_post.slug,
        excerpt=blog_post.excerpt,
        content=blog_post.content,
        keywords=", ".join(blog_post.keywords),
    )

    messages = [
        SystemMessage(content="You are a helpful evaluator of marketing content."),
        HumanMessage(content=prompt_text)
    ]

    response = llm(messages)
    return parser.parse(response.content)

In [79]:
blog_post_evaluation = evaluate_blog_post(blog_post)

In [81]:
print("Evaluation for our blog post:\n")
print(f"SEO Optimization: {blog_post_evaluation.seo_optimization}")
print(f"Clickability: {blog_post_evaluation.clickability}")
print(f"Readability: {blog_post_evaluation.readability}")
print(f"Audience Fit: {blog_post_evaluation.audience_fit}")
print(f"Content Quality: {blog_post_evaluation.content_quality}")
print(f"Comments: {blog_post_evaluation.comments}")

Evaluation for our blog post:

SEO Optimization: 4
Clickability: 5
Readability: 5
Audience Fit: 5
Content Quality: 4
Comments: The blog post effectively uses relevant keywords related to brunch and charity, enhancing its SEO potential. The title and excerpt are compelling and emotionally engaging, encouraging clicks. The structure is clear and well-formatted, making it easy to read. However, the content could benefit from more specific details about the charities involved to enhance engagement.


### 1e. Create an email blast draft out of the blog post.

Now let's create a draft of an email blast out of the blog post and the proposed idea.

In [88]:
# optional code snippet to reload the models module if the
# below import doesn't work. Jupyter notebooks can cache
# module imports.
import importlib
import marketing_agent_examples.models

# Force reload the module
importlib.reload(marketing_agent_examples.models)

<module 'marketing_agent_examples.models' from '/Users/mark/Documents/work/demos/ai_agent_learning_examples/marketing_agent_examples/models.py'>

In [89]:
from marketing_agent_examples.models import EmailBlastDraft

parser = PydanticOutputParser(pydantic_object=EmailBlastDraft)

email_blast_instructions = parser.get_format_instructions()


In [86]:
email_blast_draft_prompt = PromptTemplate(
    template="""
        You are an email marketing expert creating a launch email for a brunch restaurant's new campaign.

        You are provided with:
        <idea>
        Name: {idea_name}
        Audience: {audience}
        Campaign Message: {campaign_message}
        Concept: {concept}
        </idea>

        <blog_post>
        Title: {title}
        Excerpt: {excerpt}
        Content: {content}
        Keywords: {keywords}
        </blog_post>

        Write a short, emotionally engaging email blast targeted at the given audience. It should:
        - Grab attention in the subject line and preview text
        - Use a warm, persuasive tone that matches the audience
        - Summarize the blog post clearly and concisely
        - Lead to a strong call to action (CTA)
        - Be optimized for both desktop and mobile readers

        {format_instructions}

        Also include an explanation of why this CTA was chosen for this audience and what kind of behavioral response is expected.
""",
    input_variables=["idea_name", "audience", "campaign_message", "concept", "title", "excerpt", "content", "keywords"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [90]:
def generate_email_blast_draft(idea: ProposedIdea, blog_post: BlogPost) -> EmailBlastDraft:
    prompt_text = email_blast_draft_prompt.format(
        idea_name=idea.idea,
        audience=idea.audience,
        campaign_message=idea.campaign_message,
        concept=idea.concept,
        title=blog_post.title,
        excerpt=blog_post.excerpt,
        content=blog_post.content,
        keywords=", ".join(blog_post.keywords)
    )

    messages = [
        SystemMessage(content="You are a skilled marketing copywriter and strategist."),
        HumanMessage(content=prompt_text)
    ]

    response = llm(messages)
    return parser.parse(response.content)

In [91]:
email_blast_draft: EmailBlastDraft = generate_email_blast_draft(
    idea=best_idea,
    blog_post=blog_post
)

In [98]:
print("Email blast draft:\n")
print(f"Subject line: {email_blast_draft.subject_line}")
print(f"Preview text: {email_blast_draft.preview_text}")
print(f"Body: {email_blast_draft.body}")
print(f"Call to action: {email_blast_draft.call_to_action}")
print(f"Explanation: {email_blast_draft.explanation}")

Email blast draft:

Subject line: Brunch with a Cause: Eat, Celebrate, and Give Back!
Preview text: Join us for a delicious brunch that supports local charities. Gather your friends for a good cause!
Body: <p>Dear Brunch Lovers,</p> <p>Are you ready to make your brunch plans even more meaningful? Join us for our exciting <strong>Brunch with a Cause</strong> events, where every bite helps support local charities!</p> <p>Gather your friends and indulge in our all-you-can-eat buffet filled with mouthwatering brunch favorites. Each month, a portion of the proceeds goes directly to a different local charity, allowing you to enjoy a delightful meal while making a positive impact in our community.</p> <p>Whether you're celebrating a special milestone or just enjoying a casual get-together, our brunch is the perfect way to connect with friends and give back. Plus, you'll learn about the amazing work our featured charities are doing!</p> <p>Mark your calendars and check our website for the next

### 1f. Evaluate the email blast draft.

Now that we have an email blast draft, let's ask the LLM to evaluate the draft and score it.

Let's score our email draft on these criteria:

| Criterion                       | Description                                                            |
| ------------------------------- | ---------------------------------------------------------------------- |
| **Subject Effectiveness (0–5)** | Is the subject line engaging, clear, and likely to get opened?         |
| **Preview Quality (0–5)**       | Does the preview text create curiosity and complement the subject?     |
| **Message Clarity (0–5)**       | Is the main content easy to understand and well-structured?            |
| **CTA Strength (0–5)**          | Is the call-to-action clear, relevant, and motivating?                 |
| **Tone Fit (0–5)**              | Does the tone align with the intended audience from the original idea? |
| **Overall Comments**            | Short notes on strengths and weaknesses                                |


In [99]:
# optional code snippet to reload the models module if the
# below import doesn't work. Jupyter notebooks can cache
# module imports.
import importlib
import marketing_agent_examples.models

# Force reload the module
importlib.reload(marketing_agent_examples.models)

<module 'marketing_agent_examples.models' from '/Users/mark/Documents/work/demos/ai_agent_learning_examples/marketing_agent_examples/models.py'>

In [100]:
from marketing_agent_examples.models import EmailBlastDraftEvaluation

parser = PydanticOutputParser(pydantic_object=EmailBlastDraftEvaluation)

email_blast_draft_evaluation_instructions = parser.get_format_instructions()

In [101]:
email_blast_draft_evaluation_prompt = PromptTemplate(
    template="""
        You are a senior email marketing strategist evaluating the quality of a marketing email blast.

        Evaluate the email based on the following criteria (0-5 scale):

        - **Subject Effectiveness**: Is the subject line likely to drive opens?
        - **Preview Quality**: Does the preview text complement the subject and generate curiosity?
        - **Message Clarity**: Is the message clear, persuasive, and well-structured?
        - **CTA Strength**: Is the call-to-action obvious, relevant, and likely to convert?
        - **Tone Fit**: Does the tone match the target audience and campaign intent?

        Also provide 2-3 sentences of overall comments on strengths and areas for improvement.

        Here is the email blast to evaluate:

        Subject: {subject_line}  
        Preview: {preview_text}  
        Body: {body}  
        Call to Action: {cta}  
        Explanation: {explanation}

        {format_instructions}
""",
    input_variables=["subject_line", "preview_text", "body", "cta", "explanation"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [102]:
def evaluate_email_blast_draft(email_blast: EmailBlastDraft) -> EmailBlastDraftEvaluation:
    prompt_text = email_blast_draft_evaluation_prompt.format(
        subject_line=email_blast.subject_line,
        preview_text=email_blast.preview_text,
        body=email_blast.body,
        cta=email_blast.call_to_action,
        explanation=email_blast.explanation
    )

    messages = [
        SystemMessage(content="You are a helpful evaluator of email marketing content."),
        HumanMessage(content=prompt_text)
    ]

    response = llm(messages)
    return parser.parse(response.content)

In [103]:
email_blast_draft_evaluation: EmailBlastDraftEvaluation = evaluate_email_blast_draft(email_blast_draft)

In [104]:
print("Evaluation for our email blast draft:\n")
print(f"Subject Effectiveness: {email_blast_draft_evaluation.subject_effectiveness}")
print(f"Preview Quality: {email_blast_draft_evaluation.preview_quality}")
print(f"Message Clarity: {email_blast_draft_evaluation.message_clarity}")
print(f"CTA Strength: {email_blast_draft_evaluation.cta_strength}")
print(f"Tone Fit: {email_blast_draft_evaluation.tone_fit}")
print(f"Comments: {email_blast_draft_evaluation.comments}")

Evaluation for our email blast draft:

Subject Effectiveness: 4
Preview Quality: 4
Message Clarity: 5
CTA Strength: 4
Tone Fit: 5
Comments: The email effectively captures attention with a compelling subject line and engaging preview text that highlights the charitable aspect of the event. The message is clear and well-structured, making it easy for readers to understand the purpose and details of the brunch. However, the CTA could be slightly more urgent to enhance conversion potential.


### 1g. Create social media posts.

Now, given our email blast draft, blog post, and the idea for the campaign, let's draft some social media posts.



In [105]:
# optional code snippet to reload the models module if the
# below import doesn't work. Jupyter notebooks can cache
# module imports.
import importlib
import marketing_agent_examples.models

# Force reload the module
importlib.reload(marketing_agent_examples.models)

<module 'marketing_agent_examples.models' from '/Users/mark/Documents/work/demos/ai_agent_learning_examples/marketing_agent_examples/models.py'>

In [106]:
from marketing_agent_examples.models import SocialMediaPost, SocialMediaPostsWrapper

parser = PydanticOutputParser(pydantic_object=SocialMediaPostsWrapper)

social_media_posts_instructions = parser.get_format_instructions()

In [107]:
social_media_posts_prompt = PromptTemplate(
    template="""
        You are a social media strategist creating platform-specific posts for an American-style brunch restaurant.

        You are given:
        <idea>
        Name: {idea_name}
        Audience: {audience}
        Message: {campaign_message}
        Concept: {concept}
        </idea>

        <blog_post>
        Title: {title}
        Excerpt: {excerpt}
        Content: {content}
        Keywords: {keywords}
        </blog_post>

        <email_blast>
        Subject: {subject_line}
        Preview: {preview_text}
        Body: {body}
        Call to Action: {call_to_action}
        </email_blast>

        Create {num_posts} social media posts that are:
        - Optimized for either Facebook or Instagram
        - Tailored to the campaign audience
        - Emotionally compelling, easy to skim, and visually suggestive
        - Short enough for quick consumption
        - Include relevant and popular hashtags
        - Include the *intended ad targeting audience* (e.g., young professionals in urban areas, families with kids, health-conscious millennials, etc.)

        Output format:
        {format_instructions}
""",
    input_variables=[
        "idea_name", "audience", "campaign_message", "concept",
        "title", "excerpt", "content", "keywords",
        "subject_line", "preview_text", "body", "call_to_action",
        "num_posts"
    ],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [109]:
def generate_social_media_posts(
    idea: ProposedIdea,
    blog_post: BlogPost,
    email_blast: EmailBlastDraft,
    num_posts: int = 10
):
    prompt_text = social_media_posts_prompt.format(
        idea_name=idea.idea,
        audience=idea.audience,
        campaign_message=idea.campaign_message,
        concept=idea.concept,
        title=blog_post.title,
        excerpt=blog_post.excerpt,
        content=blog_post.content,
        keywords=", ".join(blog_post.keywords),
        subject_line=email_blast.subject_line,
        preview_text=email_blast.preview_text,
        body=email_blast.body,
        call_to_action=email_blast.call_to_action,
        num_posts=num_posts
    )

    messages = [
        SystemMessage(content="You are a professional social media copywriter."),
        HumanMessage(content=prompt_text)
    ]

    response = llm(messages)
    return parser.parse(response.content)


In [110]:
social_media_posts: SocialMediaPostsWrapper = generate_social_media_posts(
    idea=best_idea,
    blog_post=blog_post,
    email_blast=email_blast_draft,
    num_posts=10
)

In [111]:
print("Social media posts:\n")
for i, post in enumerate(social_media_posts.posts):
    print(f"Post {i+1}:")
    print(f"Platform: {post.platform}")
    print(f"Content: {post.content}")
    print(f"Hashtags: {', '.join(post.hashtags)}")
    print(f"Intended Audience: {post.intended_audience}")
    print("="*50)

Social media posts:

Post 1:
Platform: Instagram
Content: 🥂 Gather your brunch crew and join us for our monthly **Brunch with a Cause**! Enjoy an all-you-can-eat buffet while supporting local charities. Every bite makes a difference! 💖 #BrunchWithACause #SupportLocal #BrunchGoals
Hashtags: #BrunchWithACause, #SupportLocal, #BrunchGoals, #CommunityLove, #EatForACause
Intended Audience: Social brunch groups celebrating milestones or gathering casually
Post 2:
Platform: Facebook
Content: 🌟 Celebrate friendship and give back at our **Brunch with a Cause**! Join us for delicious food and support local charities. Together, we can make a difference! 🍽️❤️ #BrunchForACause #CommunitySpirit
Hashtags: #BrunchForACause, #CommunitySpirit, #GoodEats, #CharityBrunch
Intended Audience: Social brunch groups celebrating milestones or gathering casually
Post 3:
Platform: Instagram
Content: 🥞 Fluffy pancakes, savory omelets, and a cause worth supporting! Join us for our **Brunch with a Cause** and enjoy a

Now we've been able to create social media posts.

### 1h. Evaluate the social media posts.

### 1i. Chaining it all together.

Let's now chain all of these responses together.

### 1j. Takeaways and possible next steps.

## 2. Parallelization

(take the steps from above and pick some to rewrite or redo to include parallelization, and why to do parallelization in the first place. Should explain why parallelization is appropriate in some cases).

## 3. Routing

(take the steps from above and pick some to rewrite or redo to include routing, and why to do routing in the first place. Should explain why routing is appropriate in some cases).

## 4. Orchestrator-subagents

## 5. Evaluator-optimizer

Note: an example could be having an agent rewrite posts (IDK if that's best here or somewhere else).

## What's next?
(suggestions for how to improve what we've built, e.g., evals, how to optimize, etc.)

(could put this in a different Jupyter notebook TBH? Something about follow-ups? IDK if best here, since this would likely be generic across all workflows)